In [1]:
!pip install pyspark


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [19]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [20]:
from pyspark.sql import SparkSession

In [21]:
spark = SparkSession.builder.appName("IR-Inverted-index").config("spark.executor.memory", "1g").config("spark.executor.cores", 2).getOrCreate()

In [22]:
import zipfile

In [11]:
zip_file_path = "content/data2.zip"
extract_folder = "/content/extracted_files/"

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

OSError: [Errno 30] Read-only file system: '/content'

In [9]:
pwd

'/Users/samroadie/Downloads/Pavana_Chalamarla_HW3_8495176896'

In [23]:
txt_rdd = spark.sparkContext.wholeTextFiles("/content/data/fulldata/*.txt")

In [24]:
txt_df = txt_rdd.toDF()

Py4JJavaError: An error occurred while calling o40.partitions.
: org.apache.hadoop.mapreduce.lib.input.InvalidInputException: Input Pattern file:/content/data/fulldata/*.txt matches 0 files
	at org.apache.hadoop.mapreduce.lib.input.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:340)
	at org.apache.hadoop.mapreduce.lib.input.FileInputFormat.listStatus(FileInputFormat.java:279)
	at org.apache.spark.input.WholeTextFileInputFormat.setMinPartitions(WholeTextFileInputFormat.scala:52)
	at org.apache.spark.rdd.WholeTextFileRDD.getPartitions(WholeTextFileRDD.scala:54)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:294)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:290)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:294)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:290)
	at 

In [9]:
txt_df.show(2)

+--------------------+--------------------+
|                  _1|                  _2|
+--------------------+--------------------+
|file:/content/ext...|5722018435\tWe re...|
|file:/content/ext...|5722018436\tIf yo...|
+--------------------+--------------------+
only showing top 2 rows



In [10]:
from pyspark.sql.functions import col, regexp_replace, lower,expr,explode, split,concat

In [11]:
txt_df = txt_df.withColumn("text_cleaned", regexp_replace("_2", "(?<=\t)", ""))

In [12]:
txt_df.show(2)

+--------------------+--------------------+--------------------+
|                  _1|                  _2|        text_cleaned|
+--------------------+--------------------+--------------------+
|file:/content/ext...|5722018435\tWe re...|5722018435\tWe re...|
|file:/content/ext...|5722018436\tIf yo...|5722018436\tIf yo...|
+--------------------+--------------------+--------------------+
only showing top 2 rows



In [13]:
txt_df = txt_df.withColumn("doc_id", expr("split(text_cleaned, '\t')[0]"))

txt_df = txt_df.withColumn("final_text", expr("lower(regexp_replace(trim(substring(text_cleaned, length(split(text_cleaned, '\t')[0])+2)), '[^a-zA-Z\\s]', ' '))"))

In [14]:
df_exploded = txt_df.select("doc_id", explode(split(col("final_text"), "\\s+")).alias("word"))

df_counts = df_exploded.groupBy("word", "doc_id").count()

df_unigram_index = df_counts.groupBy("word").pivot("doc_id").sum("count").fillna(0)

In [ ]:
df_unigram_index.show(10)

+---------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+
|     word|5722018435|5722018436|5722018437|5722018438|5722018439|5722018440|5722018441|5722018442|5722018443|5722018444|5722018445|5722018446|5722018447|5722018448|572201844

In [15]:
from pyspark.sql.functions import concat_ws

df_unigrams = df_exploded
df_unigrams = df_unigrams.where(col("word") != "")

unigram_index = df_unigrams.groupBy("word", "doc_id").count()

unigram_index_formatted = unigram_index.groupBy("word").agg(expr("collect_list(concat(doc_id, ':', count)) as doc_counts"))

output = unigram_index_formatted.withColumn("unigram_data", concat_ws(' ', col("word"), expr("concat_ws(' ', doc_counts)")))

output.select("unigram_data").repartition(1).write.mode("overwrite").text("output_unigram")

In [1]:
!pip install pyspark


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("IR-Inverted-index").config("spark.executor.memory", "1g").config("spark.executor.cores", 2).getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/20 12:31:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/20 12:31:49 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [5]:
import zipfile

In [11]:
zip_file_path = "content/data2.zip"
extract_folder = "/content/extracted_files/"

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

OSError: [Errno 30] Read-only file system: '/content'

In [9]:
pwd

'/Users/samroadie/Downloads/Pavana_Chalamarla_HW3_8495176896'

In [12]:
txt_rdd = spark.sparkContext.wholeTextFiles("/content/data/fulldata/*.txt")

In [13]:
txt_df = txt_rdd.toDF()

Py4JJavaError: An error occurred while calling o30.partitions.
: org.apache.hadoop.mapreduce.lib.input.InvalidInputException: Input Pattern file:/content/data/fulldata/*.txt matches 0 files
	at org.apache.hadoop.mapreduce.lib.input.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:340)
	at org.apache.hadoop.mapreduce.lib.input.FileInputFormat.listStatus(FileInputFormat.java:279)
	at org.apache.spark.input.WholeTextFileInputFormat.setMinPartitions(WholeTextFileInputFormat.scala:52)
	at org.apache.spark.rdd.WholeTextFileRDD.getPartitions(WholeTextFileRDD.scala:54)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:294)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:290)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:294)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:290)
	at 

In [9]:
txt_df.show(2)

+--------------------+--------------------+
|                  _1|                  _2|
+--------------------+--------------------+
|file:/content/ext...|5722018435\tWe re...|
|file:/content/ext...|5722018436\tIf yo...|
+--------------------+--------------------+
only showing top 2 rows



In [10]:
from pyspark.sql.functions import col, regexp_replace, lower,expr,explode, split,concat

In [11]:
txt_df = txt_df.withColumn("text_cleaned", regexp_replace("_2", "(?<=\t)", ""))

In [12]:
txt_df.show(2)

+--------------------+--------------------+--------------------+
|                  _1|                  _2|        text_cleaned|
+--------------------+--------------------+--------------------+
|file:/content/ext...|5722018435\tWe re...|5722018435\tWe re...|
|file:/content/ext...|5722018436\tIf yo...|5722018436\tIf yo...|
+--------------------+--------------------+--------------------+
only showing top 2 rows



In [13]:
txt_df = txt_df.withColumn("doc_id", expr("split(text_cleaned, '\t')[0]"))

txt_df = txt_df.withColumn("final_text", expr("lower(regexp_replace(trim(substring(text_cleaned, length(split(text_cleaned, '\t')[0])+2)), '[^a-zA-Z\\s]', ' '))"))

In [14]:
df_exploded = txt_df.select("doc_id", explode(split(col("final_text"), "\\s+")).alias("word"))

df_counts = df_exploded.groupBy("word", "doc_id").count()

df_unigram_index = df_counts.groupBy("word").pivot("doc_id").sum("count").fillna(0)

In [ ]:
df_unigram_index.show(10)

+---------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+
|     word|5722018435|5722018436|5722018437|5722018438|5722018439|5722018440|5722018441|5722018442|5722018443|5722018444|5722018445|5722018446|5722018447|5722018448|572201844

In [15]:
from pyspark.sql.functions import concat_ws

df_unigrams = df_exploded
df_unigrams = df_unigrams.where(col("word") != "")

unigram_index = df_unigrams.groupBy("word", "doc_id").count()

unigram_index_formatted = unigram_index.groupBy("word").agg(expr("collect_list(concat(doc_id, ':', count)) as doc_counts"))

output = unigram_index_formatted.withColumn("unigram_data", concat_ws(' ', col("word"), expr("concat_ws(' ', doc_counts)")))

output.select("unigram_data").repartition(1).write.mode("overwrite").text("output_unigram")